# Discrete Choice Analysis: micro-econometrics and machine learning approaches

## `Lab session 1`<br> `Artificial Neural Networks`

**Delft University of Technology**<br>
**July 2022**<br>
**Instructor:** Sander van Cranenburgh <br>
**TAs:**  Francisco Garrido Valenzuela & Lucas Spierenburg <br>

# `Instructions`

**Lab sessions aim to:**<br>
* Show and reinforce how models and ideas presented in class are put to practice.<br>
* Help you gather hands-on machine learning skills.<br>

**Lab sessions are:**<br>
* Learning environments where you work with Jupyter notebooks and where you can get support from TAs and fellow students.<br> 
* Not graded and do not have to be submitted. 

# `Workspace set-up`
**Option 1: Google Colab**<br>
Uncomment the following cell if you are running this notebook on Colab

In [ ]:
#!rm -rf /content/ML_approaches_for_discrete_choice_analysis
#!git clone https://github.com/TPM34A/ML_approaches_for_discrete_choice_analysis
#!pip install -r ML_approaches_for_discrete_choice_analysis/requirements_colab.txt
#!cp -R "/content/ML_approaches_for_discrete_choice_analysis/Lab sessions/data" /content/data
#num_iters = 250

**Option 2: Local environment**<br>
Uncomment the following cell if you are running this notebook on your local environment, to install all dependencies on your Python version.

In [ ]:
#!pip install -r requirements_local.txt

# `Application: Swiss mode choice` <br>
In this lab session we will use ANN to predict mode choice behaviour. Being able to predict model choice behaviour is highly policy relevant. Policy makers use it to develop policies aimed at reducing externalities associated with travel behaviour, such as e.g. fuel levies, subsidization of public transport, speed limits. Additionally, they use it for planning transport infrastructure. Based on travel demand forecast e.g. roads are widened or new train connections are built. Traditionally, mode choice predictions are based on classic econometric tools. Recently, the high prediction performance of ML methods appeal researchs to use ML for model choice predictions too. In this lab session you will develop ANNs to predict mode behaviour and assess its performance<br>


**Learning objectives**. After completing the following exercises you will be able to: <br>
1. Prepare (choice) data for training Artificial Neural Networks
2. Train MultiLayerPerceptron (MLP) - a praticular type of neural network - for a classification task<br>
3. Tune the hyperparameter and network architectures to improve the model performance<br>
4. Assess the performance of of competing models, based on various performance measures, including confusion matrices, and Precision, Recall, F1 scores and Matthew's coefficient<br>


#### `Organisation`
This lab session comprises **`6`** parts:
1. Preparing (choice) data for training Artificial Neural Networks
2. Training a MultiLayerPerceptron (MLP) neural network
3. Using Early stopping to avoid overfitting
4. Using k-fold cross validation to evaluate generalisation performance
5. Tuning hyperparameter
6. Evaluating performance of trained models

and comprises **`7`** exercises.

In [ ]:
# Import required Python packages and modules
import os
import pandas as pd
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

# Import selected functions and classes from Python packages
from os import getcwd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import ConfusionMatrixDisplay, log_loss, matthews_corrcoef, make_scorer, classification_report

# Setting
pd.set_option('display.max_columns', None)

### **1. Preparing (choice) data for training Artificial Neural Networks**
To prepare the data set, we will:<br>
    1.1 **Load** the data set<br>
    1.2 **Inspect** and **Clean** the data set<br>
    1.3 **Discover and visualise** the data <br>

**1.1 Load the choice data**

In [ ]:
# Get the current working directory
datafile_path = os.path.join(getcwd(),'data','')
print(datafile_path)

In [ ]:
# Load mode choice data into a pandas DataFrame
df = pd.read_csv(datafile_path + 'swissmetro.dat',sep = '\t')

**1.2 Inspect and clean the data**<br>
Before starting to analyse your data, make sure you understand what features are in your data.<br>
Therefore, it is highly recommended to look at the description of the data set. [Click here](https://github.com/TPM34A/Admin_2022/blob/main/lab%20sessions/lab_ex02/data/CS_SwissmetroDescription.pdf) to open the pdf.

In [ ]:
# Inspect the range and distribution statistics 
df.describe()

# Inspect the data types in the df
# df.info()

In [ ]:
# Data cleaning

# Only keep data for purposes 'Commute" and "Business"
df.drop(df[(df.PURPOSE != 1) & (df.PURPOSE != 3)].index, inplace=True) 

# Drop rows with unknown choices (CHOICE == 0)
df.drop(df[df.CHOICE == 0].index, inplace=True) 

# In case of missing values, we replace them with 0
df.fillna(0, inplace = True) 

**1.3 Discovering and visualising the data**<br>
Before starting to analyse your data with models, it is advisable to start with some **descriptive analyses**.<br>
Therefore is recommended to look first at the distribution and correlations of key feature in the data.

### ``Exercise 1: Is the data set (im)balanced?``
`A` Create a histogram showing how often TRAIN, SM and CAR are chosen. Do not forget to add labels to the columns<br>
`B` Interpret the the histogram. Is the data set imbalanced? <br>

$\color{green}{\text{Add your answers here}}$

In [ ]:
# Add your code here

In [ ]:
# ANSWERS
# A) 
# B) 

### ``Exercise 2: Explore correlations between features``
`A` Create a heatmap to identify what features particulalrly correlate with the **CHOICE**<br>
`B` Identify the features that strongly correlate (corr >0.9) (if any). Do they make sense? <br>
Hint: to do so you will need the description of the features [Click here](https://github.com/TPM34A/Admin_2022/blob/main/lab%20sessions/lab_ex02/data/CS_SwissmetroDescription.pdf) to open the pdf.

$\color{green}{\text{Add your answers here}}$

In [ ]:
# Add your code here

In [ ]:
# ANSWERS
# A) 
# B)

### **2. Training a MultiLayerPerceptron (MLP) neural network**
To train Artifical Neural Networks we take the following steps:<br>
    2.1 **Scaling** the features<br>
    2.2 **Splitting** the data in a training and test data set<br>
    2.3 **Creating MLP** object<br>
    2.4 **Training the MLP** on the train data<br>
    2.5 **Evaluating** the performance


**2.1 Scaling the features**<br>
To efficiently train ANNs it strongly recommended to **scale** (a.k.a. normalise) the features. There are serval ways to scale your data. A commonly used scaler of `sk-learn` is called 'StandardScaler'. This scaler normalises the variance and shift the location of the distribution to zero, see https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [ ]:
# Creates the list of features that need to be scaled
# Importantly, this excludes the availabilities and the choice
features2scale = ['PURPOSE', 'FIRST', 'TICKET','WHO', 'LUGGAGE', 'AGE','MALE', 'INCOME', 'GA','ORIGIN', 'DEST',
                 'TRAIN_TT', 'TRAIN_CO', 'TRAIN_HE',
                 'SM_TT', 'SM_CO','SM_HE', 'SM_SEATS',
                 'CAR_TT', 'CAR_CO']

# Initiate scaler object & fit to data
scaler = StandardScaler()  
scaler.fit(df.loc[:,features2scale])  

# Create new dataframe X_scaled containg the scaled features AND the (unscaled) ones: availabilities, Group, Survey, Choice
X_scaled = df.copy()
X_scaled.at[:, features2scale] = scaler.transform(df.loc[:,features2scale])

**2.2 Splitting the data in a train set and a test set**<br>
Training ML models always involves a **train** and a **test** data set. The train set is used to update the weights of the model. Tha test set is used to evaluate the **generalisation performance** of the model.

In [ ]:
# Create the list of features that we want to use in the model
features = ['PURPOSE', 'FIRST', 
            'TICKET', 'WHO', 'LUGGAGE', 'AGE', 
            'MALE', 'INCOME', 'GA', 'ORIGIN', 
            'DEST', 'TRAIN_TT', 'TRAIN_CO', 
            'TRAIN_HE', 'SM_TT', 'SM_CO', 
            'SM_HE', 'SM_SEATS', 'CAR_TT', 'CAR_CO',
            'TRAIN_AV', 'SM_AV', 'CAR_AV']

# Create the target
Y = df['CHOICE']

# Split the data using sk-learn's `train_test_split` function
# Note that we use 60% for training and 40% for testing
# Note that we set the random_state, in order to replicate results later (do not change) 
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled[features], Y, random_state = 12345, test_size = 0.4)

**2.3 Creating the MLP object**<br>
A MultiLayerPerceptron (MLP) is a fully-connected feed-forward neural network. We create the MLP using `sk-learn's` MLPClassifier function. 

In [ ]:
# Create 'plain vanilla' MLP object
# Declare the number of layers and nodes per layer
# layers = (a,b) means two layer with a nodes in the 1st hidden layer and b nodes in the 2nd hidden layer
layers = (10)

# Define MLP architecture, optimiser and hyperparameters:
# We use Adam optimiser
# We use the learning rate to 0.001
# We use L2 regularisation of 0.1
# We use a batch size of 250 observations
# We use tanhg activation (transfer function)
# We set the max number of epochs to 2000
mlp = MLPClassifier(hidden_layer_sizes = layers, solver='adam', learning_rate_init = 0.001, alpha=0.1, batch_size=250, activation = 'tanh', max_iter = 2000) 

**2.4 Training the MLP**<br>
To train the MLP we use the '.fit' function.  

In [ ]:
# Train the MLP using the train data.
mlp.fit(X_train, Y_train)

In [ ]:
# It is good practice to look at the learning curve, to see how much the model has improved from the initial point,
# and how is has evolved over epochs.
fig, ax = plt.subplots(figsize = (16,8))
plt.plot(mlp.loss_curve_)
ax.set_xlabel('Epoch')
ax.set_ylabel('Cross entropy loss')
ax.grid(True,linewidth = 0.5)
ax.set_ylim(0.0,1.2)
ax.set_xlim(0,mlp.n_iter_)
ax.set_title(f'Cross entropy loss on the TRAINING DATA. \nBest CE = {mlp.loss_:4.3f}')
# The plot illustrates that the training stops when the training loss does no longer improve more that a given tolerance (e.g. 1e-6), or reaches max_iter

**2.5 Evaluating the model performance** <br>
Machine learning researchers usually evaluate the performance of a classification model using the cross-entropy. A lower cross entrop indicates a better model. In contrast, choice modellers usually look at the Log-likelihood (LL) and the rho-square. A high LL and rho-square indicate a better model. <br>
As there is no standard function that outputs the evaluation metrics of both disciplines, below we create our **own evaluation function**.

In [ ]:
# We create an eavluation function that returns key evaluation metrics: LL, LL0, cross_entropy, rho_sq
# To compute these performance metrics, the function takes as inputs:
#   - the predicted probabilities (prob)
#   - the choices (Y)
#   - the availabilities (AV)
def eval_performance(prob,Y,AV):
    
    # Calculate the likelihood of the data given the model
    LL = np.sum(np.log(np.sum(prob*Y,axis=1)))

    # Calculate the Null-loglikelihood
    LL0 = np.sum(np.log(np.divide(1,np.sum(AV,axis=1))))

    # Calculate cross-entropy
    cross_entropy =  -LL/len(AV)
    
    # Calculate the rho_sq
    rho_sq = 1 -(LL/LL0)
    return LL, LL0, cross_entropy, rho_sq

In [ ]:
# Let's use our function to evaluate and compare the performance of our MLP on the training and test data sets
eval_train = eval_performance(mlp.predict_proba(X_train),np.transpose([Y_train ==1,Y_train ==2,Y_train ==3]), X_train[['TRAIN_AV','SM_AV','CAR_AV']])
eval_test  = eval_performance(mlp.predict_proba(X_test), np.transpose([Y_test  ==1,Y_test  ==2,Y_test  ==3]), X_test[['TRAIN_AV','SM_AV','CAR_AV']])

# Print the results
print('Model performance of the plain vanilla MLP:')
pd.options.display.float_format = '{:,.2f}'.format
results = pd.DataFrame({'data set':     ['Train','Test'],
                        'LL':           [eval_train[0], eval_test[0]],
                        'LL0':          [eval_train[1], eval_test[1]],
                        'cross_entropy':[eval_train[2], eval_test[2]],
                        'rho_sq':       [eval_train[3], eval_test[3]]})
print(results.T)

To get a sense of how well this plain vanilla MLP performs, below are the same performance metrics for a RUM-MNL model printed.

In [ ]:
print('\nModel performance of plain vanilla linear-additive RUM-MNL model:')
print('LL\t\t -5331.252 \nLL0\t\t -6964.663\ncross_entropy\t 0.79 \nrho_sq\t\t 0.23')

### ``Exercise 3: Does using more nodes improve the model performance?``
`A` Calculate the number of weights consumed by the current MLP with 10 hidden nodes in 1 hidden layer. <br>
`B` Retrain your model several times with {10,30,60,90} nodes. Report the cross-entropy performance on the train and test data sets.<br>
`C` Does increasing the number of nodes lead to a lower cross-entropy on the train and or test set? What is happening?<br>
`D` Suppose you would have many more choice observations from this survey. Would that enable you to develop a much better model with a cross entropy performance of say <0.10 (on the test set)? Explain your answer. <br>

$\color{green}{\text{Add your answers here}}$

In [ ]:
# Add your code here

In [ ]:
# ANSWERS
# A)
# B)
# C)
# D)

### **3. Using Early stopping to avoid overfitting**
Early stopping refers to a technique that stops the training of the network when the performance on the test data set no longer improves. Thereby, early stopping avoids the model to overfit the data. It essetially stops the training before the model can overfit.  

In [ ]:
# Note this is the same 'plain vanilla' MLP, but now we set early_stopping = True
# A validation_fraction is added. This fraction is the proportion of training data to set aside as validation set for early stopping. This data set is used to determine when to stop. 
# The training stops when the performance on the validation data set does not improve for n_iter_no_change in a row
# We use the MLP with 10 nodes and one hidden layer again.
layers = (10)
n_iter_no_change = 10
mlp_early_st = MLPClassifier(hidden_layer_sizes = layers, solver='adam', learning_rate_init = 0.001, alpha=0.1, batch_size=250, activation = 'tanh', max_iter = 2000, early_stopping=True, n_iter_no_change = n_iter_no_change,validation_fraction = 0.25) 

# Train the MLP using the train data
# Note that we use df.values here. This is due to a small bug in sk-learn. Without .values sk-learn still works, but prompts some warnings
mlp_early_st.fit(X_train.values, Y_train.values)

In [ ]:
# It is good practice to look at the learning curve, to see how much the model has improved from the initial point,
# and how it has evolved over the epochs.
# The learning curve plot also illustrates well what early stopping does.
fig, ax = plt.subplots(figsize = (16,8))
plt.plot(mlp.loss_curve_,label=f'MLP with {mlp.hidden_layer_sizes} hidden nodes, WITHOUT early stopping')
plt.plot(mlp_early_st.loss_curve_,label=f'MLP with {mlp_early_st.hidden_layer_sizes} hidden nodes, WITH early stopping')
ax.set_xlabel('Epoch')
ax.set_ylabel('Cross entropy loss')
ax.grid(True,linewidth = 0.5)
ax.set_ylim(0.0,1.2)
ax.set_xlim(0,mlp.n_iter_)
ax.set_title('Effect of early stopping')
ax.legend()

In [ ]:
# Let's use our evaluation function again to evaluate performance of the MLP with early stopping on the training and test data set
eval_train_early_st = eval_performance(mlp_early_st.predict_proba(X_train.values),np.transpose([Y_train ==1,Y_train ==2,Y_train ==3]), X_train[['TRAIN_AV','SM_AV','CAR_AV']])
eval_test_early_st  = eval_performance(mlp_early_st.predict_proba(X_test.values), np.transpose([Y_test  ==1,Y_test  ==2,Y_test  ==3]), X_test[['TRAIN_AV','SM_AV','CAR_AV']])

pd.options.display.float_format = '{:,.2f}'.format
results_early_st = pd.DataFrame({'data set':     ['Train','Test'],
                        'LL':           [eval_train_early_st[0], eval_test_early_st[0]],
                        'LL0':          [eval_train_early_st[1], eval_test_early_st[1]],
                        'cross_entropy':[eval_train_early_st[2], eval_test_early_st[2]],
                        'rho_sq':       [eval_train_early_st[3], eval_test_early_st[3]]})
print(results_early_st.T)

### ``Exercise 4: Can we overfit with early stopping?``
`A` Did early stopping reduced overfitting? How can you see this from the printed results?<br>
`B` Try if early stoppping also helps to avoid overfitting when using more nodes, e.g. 90 nodes

$\color{green}{\text{Add your answers here}}$

In [ ]:
# Add your code here

In [ ]:
# ANSWERS
# A)
# B)

### **4. Using k-fold cross validation to evaluate generalisation performance**
k-fold cross validation is commonly used to more accurately **evaluate the generalisation performance** of a given network. It improves a simple train-test split approach in that it systematically cuts the data set in k pieces. k-fold cross validation is especially crucial when tuning hyperparameters (as we will see later).

In [ ]:
# Create MLP object (plain vanilla MLP without early stopping)
layers = 10
mlp_cv = MLPClassifier(hidden_layer_sizes = layers, solver='adam', learning_rate_init = 0.001, alpha=0.1, batch_size=250, activation = 'tanh', max_iter = 2000) 

# Create scoring function
# It is necessary to create a scoring function when working with cross_validate of sk-learn
# We set `greater_is_better` to `False` as we are minimising cross entropy loss
logloss = make_scorer(log_loss, greater_is_better = False, needs_proba = True)

# Apply cross_validate, using e.g. 5 folds
# Since we use cross-validation training takes n_folds times longer than using a train-test split
n_folds = 5
cv_results = cross_validate(mlp_cv,X_scaled[features],Y,cv = n_folds, scoring=logloss,return_train_score=True)

In [ ]:
# Plot the train and test performance in a bar plot, for each fold
fig, ax = plt.subplots(figsize=(10, 6))
x_axis = np.arange(n_folds)
ax.bar(x_axis + -0.125, -cv_results['train_score'], color = 'b', width = 0.25,label = 'Train data set')
ax.bar(x_axis +  0.125, -cv_results['test_score'], color = 'g', width = 0.25,label = 'Test data set')
ax.set_xlabel('fold #')
ax.set_ylabel('cross entropy')
ax.grid()
ax.legend()

$\color{skyblue}{\text{Hence, this cross validation analysis supports the finding that a plain vanilla MLP (i.e. without early stopping) overfits the data considerably}}$

### **5. Tuning hyperparameter**
When training MLPs (and most other ML models) there are several parameters we can **'tune'** (optimise) to improve the model's performance. The process of doing this is called **hyperparameter tuning**. Hyperparameter tuning can be done manually, but that is unwieldy. The GridSearchCV function in `sk-learn` automates the hyperparemeter tuning process. When tuning the hyperparameters, it is mandatory to use a k-fold cross validation approach. Otherwise, there is a risk of overfitting on the test set *because* the parameters can be tweaked until the estimator performs optimally on the test data set.

In [ ]:
# Do not change these lines (they ensure the code runs fast enough on colab and local, by reducing the no. iterations for colab)
try:
    num_iters 
except NameError:
    num_iters = 2000
else:
    num_iters = 250

# Create MLP object (plain vanilla MLP)
mlp_gs = MLPClassifier(activation = 'tanh', solver='adam', batch_size=250, max_iter=num_iters)

# Define the hyperparameter search space
# 'hidden_layer_sizes' defines the number of nodes and layers
# 'alpha' governs the L2 regularisation
# 'learning_rate_init' governs the learning rate.
hyperparameter_space = {
    'hidden_layer_sizes': [(10),(10,10),(30,30)],
    'alpha': [1,1e-2,1e-4],
    'learning_rate_init': [0.01,0.001,0.0001]}

# Create scoring function
logloss = make_scorer(log_loss, greater_is_better = False, needs_proba = True)

# Create the grid_search object, with using the MLP classifier
folds = 5 # Number of cross validation splits
mlp_gridsearch = GridSearchCV(mlp_gs, hyperparameter_space, n_jobs=-1, cv=folds,scoring = logloss)

In [ ]:
# Execute the training/gridsearch
# Note that this is computationally expensive! 
# It may take up to 5 minutes, since 3 x 3 x 3 = 27 models need to be trained, each with 5 folds (=135)
mlp_gridsearch.fit(X_train, Y_train)

Save your hyperparameter tuning results, so we only have to do it once.

In [ ]:
# Save your model 
filename = 'my_tuned_model.sav'
pickle.dump(mlp_gridsearch, open(filename,'wb'))

In [ ]:
# Load your model (if you have a saved model)
# mlp_gridsearch = pickle.load(open(filename,'rb'))

Visualise the hyperparameter tuning results

In [ ]:
# Collect results into a new pandas dataframe
df_gridsearch = pd.DataFrame.from_dict(mlp_gridsearch.cv_results_)
    
# Add new column with a label for the hyperparameter combinations %% NOT SURE IF THIS IS BET WAY TO DO THIS IN PYTHON   
df_gridsearch['gs_combinations'] = 'L2 = '+ df_gridsearch['param_alpha'].astype('str') + '; Learning_rate = '+ df_gridsearch['param_learning_rate_init'].astype('str') + '; Layers = ' + df_gridsearch['param_hidden_layer_sizes'].astype('str')
df_gridsearch = df_gridsearch.sort_values('rank_test_score')

# Visualise deviation in performance across hyper parameter settings
plt.figure(figsize = (16,6))
ax = sns.barplot(x = df_gridsearch.gs_combinations,y=-df_gridsearch.mean_test_score,palette="Blues_d",)
plt.xticks(rotation=90)

print('Best hyperparameters found:\t', mlp_gridsearch.best_params_)
print('Best model performance:\t\t', -mlp_gridsearch.best_score_)

### ``Exercise 5: Hyperparameter tuning``
`A` What hyperparameter turns out to be particularly impactful on the model performance?<br>
`B` Can you think of reasons why this could be the case?

$\color{green}{\text{Add your answers here}}$

In [ ]:
# Add your code here

In [ ]:
# ANSWERS
# A)
# B)

**(Re)Training the model with optimised hyperparameters**<br> 
After completing hypertuning, you know the optimal hyperparameters. <br>
Therefore, after hypertuning we always retrain the model using the optimised hyperparameters.

In [ ]:
# Create a new mlp object using the optimised hyperparameters, just using the train/test split
layers = mlp_gridsearch.best_params_['hidden_layer_sizes']
lr = mlp_gridsearch.best_params_['learning_rate_init']
alpha = mlp_gridsearch.best_params_['alpha']
mlp_gs = MLPClassifier(hidden_layer_sizes = layers, solver='adam', learning_rate_init = lr, alpha=alpha, batch_size=250, activation = 'tanh', max_iter = 2000) 

# Train the model
mlp_gs.fit(X_train,Y_train)

In [ ]:
# Let's also evaluate performance on the hypertuned model using our evaluation function
# Note we use the full data here
eval_gridsearch = eval_performance(mlp_gs.predict_proba(X_scaled[features]),np.transpose([Y ==1,Y ==2,Y ==3]), X_scaled[['TRAIN_AV','SM_AV','CAR_AV']])

pd.options.display.float_format = '{:,.2f}'.format
results_clf = pd.DataFrame({'data set': ['All data'],
                        'LL':           [eval_test[0]],
                        'LL0':          [eval_test[1]],
                        'cross_entropy':[eval_test[2]],
                        'rho_sq':       [eval_test[3]]})
print(results_clf.T)

### **6. Evaluating and comparing performances across trained models**

To evaluate the performance of a wide range is metrics are available, beyond generalisation performance.<br>
Here, we look at:<br>
i. Confusion matrix<br>
ii. Precision, Recall, and F1-score<br>
iii. Matthew's correlation coefficient

**i) Confusion matrix**<br>
Confusion matrices shows counts from predicted and actual outcomes. The counts on the diagonal are correctly classified outcomes (the model predictions and the ground true are the same). The counts on the off diagonal elements are the misclassified outcomes. Hence, the best classifier will have a confusion matrix with only diagonal elements and the rest of the elements set to zero.

In [ ]:
# Predict the choices for the test data set, using the MLP trained with early stopping and MLP with hyperparameters tuned
Y_pred_early_st  = mlp_early_st.predict(X_test.values)  # 0/1 predictions of MLP trained with early stopping
Y_pred_gs = mlp_gs.predict(X_test)                      # 0/1 predictions of MLP with hyperparameters tuned

# Show the confusion matrices, to compare the hyperparameter tuned network with the early stopping network
fig, ax = plt.subplots(2,2,figsize = (16,12))
fig.set_tight_layout(True)
ylabels = ['Train', 'SM', 'Car']
cm1 = ConfusionMatrixDisplay.from_predictions(y_true=Y_test,y_pred=Y_pred_early_st, display_labels = ylabels, normalize=None,  ax=ax[(0,0)])
cm2 = ConfusionMatrixDisplay.from_predictions(y_true=Y_test,y_pred=Y_pred_early_st, display_labels = ylabels, normalize='true',ax=ax[(1,0)])
cm3 = ConfusionMatrixDisplay.from_predictions(y_true=Y_test,y_pred=Y_pred_gs, display_labels = ylabels, normalize=None,  ax=ax[(0,1)])
cm4 = ConfusionMatrixDisplay.from_predictions(y_true=Y_test,y_pred=Y_pred_gs, display_labels = ylabels, normalize='true',ax=ax[(1,1)])

# Add titles
cm1.ax_.set_title(f'MLP with {mlp_early_st.hidden_layer_sizes} nodes \n trained with early stopping')
cm2.ax_.set_title(f'MLP with {mlp_early_st.hidden_layer_sizes} nodes \n trained with early stopping')
cm3.ax_.set_title(f'MLP with {mlp_gs.hidden_layer_sizes} nodes \n hyperparameters tuned')
cm4.ax_.set_title(f'MLP with {mlp_gs.hidden_layer_sizes} nodes \n hyperparameters tuned')

### ``Exercise 6: Model accuracy``
Accuracy is defined as the true positives over the total number of cases.<br>
`A` Manually compute the prediction accuracy of the model with early stopping and the model with hyperparameter tuning<br>
`B` For which class (Train/SM/Car) does the hyperparameter tuning improves the prediction accuracy most?<br>

$\color{green}{\text{Add your answers here}}$

In [ ]:
# Add your code here

In [ ]:
# ANSWERS
# A)
# B)

**ii) Precision, Recall, and F1**<br>
Looking at the confusion matrices, the improvements in prediction accuracy due to the hyperparameter tuning may not seem very impressive. However, one should keep in mind that 0/1 predictions are sensitive to class imbalances, which are present in these data. Moreover, accuracy can be a misleading metric for imbalanced data sets. A naive model that would simply always predict "SM" will already do quite good.

To assess the model performance in more depth, we thus must look at the predictions at the level of the classes.<br> 
Next, we compute Precision, Recall, and F1 score.<br>
* **Precision** Tells you what fraction of predictions for a given class are actually of that class.<br>
* **Recall** Tells what fraction of all observations belonging to a given class are correctly predicted as such by the model. Recall is also known as True Positive Rate (TPR), Sensitivity, Probability of Detection. <br>
* **F1 score** combines precision and recall into a single measure. Mathematically it’s the harmonic mean of precision and recall. It can be calculated as follows: <br>

In [ ]:
# To calculate the precision, recal and f1 score we conveniently use sk-learn's 'classification_report' functionality
print('Classification report for plain vanilla MLP with early stopping\n',classification_report(Y_test,Y_pred_early_st, target_names= ylabels))
print('\nClassification report for plain vanilla MLP with hyperparameters tuned\n',classification_report(Y_test,Y_pred_gs, target_names= ylabels))

**iii) Matthew's correlation coefficient**<br>
Another commonly used metric to evaluate the prediction performance while accounting for imbalances in the data set is  matthews correlation coefficient. Matthews Correlation Coefficient (MCC) is generally regarded as being one of the best measures to describe the confusion matrix of true and false positives and negatives by a **single number**, even if the classes are of very different sizes. The MCC is in essence a correlation coefficient value between -1 and +1. A coefficient of +1 represents a perfect prediction, 0 an average random prediction and -1 an inverse prediction.

In [ ]:
# Taking into account the imbalances of the data
print(f'Matthews correlation coefficient for plain Vanilla MLP with early stopping:\t {matthews_corrcoef(Y_test, Y_pred_early_st):4.3f}')
print(f'Matthews correlation coefficient for plain Vanilla MLP with hypertuning:\t {matthews_corrcoef(Y_test, Y_pred_gs):4.3f}')

### ``Exercise 7: Model precision, recall, f1 and Matthew's correlation coefficent``
`A` Compare and interpret the results from the classifications reports between the early stopping and hypertuned model.<br>
`B` Compare and interpret the results from Matthews correlation coefficient between the early stopping and hypertuned model.

$\color{green}{\text{Add your answers here}}$

In [ ]:
# Add your code here

In [ ]:
# ANSWERS
# A)
# B)